In [28]:
import os
import json
import openai
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from uuid import uuid4
from random import randint
from utils.get_openai_api_key import get_test_key
from utils.get_postgres_connection import _conn_open
from utils.load_json import load_json
from rich.console import Console


In [29]:

console = Console()

In [30]:
OPENAI_API_KEY = get_test_key()
# Set up your OpenAI API key
openai.api_key = OPENAI_API_KEY
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
console.print(OPENAI_API_KEY)

Success: .env file found with some environment variables

sk-proj-5AApBUEbDykyV5gVXqzArwMnOM7OXM-xOBbUJo2vwlD6we99fvPicSc432PCbe93EuFjmVQL1eT3BlbkFJ0t8QIry4fKSC79pjEVlIxvtOD
Lm0d76RQInE6kFUQZsbA5HcUukLnzjYmLUcLyoXKnaj7DBjIA

In [31]:
def get_embedding(text, model="text-embedding-3-small"):

    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [32]:
conn = _conn_open()
cur = conn.cursor()

Conn: <connection object at 0x000002168F8B7CD0; dsn: 'user=postgres password=xxx dbname=postgres 
host=host.docker.internal', closed: 0>

In [33]:
console = Console()
FILENAME = "test_article_elements.json"
json_data = load_json(FILENAME)
print(f"Number of items: {len(json_data)}")
encoded_image_data = None
for i in range(len(json_data)):

    if json_data[i]["type"] == "Image":
        console.print(f"[dark_orange]---------Item Number IMAGE: {i + 1}--------[/]")
        el_image_b64 = json_data[i]["metadata"]["image_base64"]

        # console.print(el_image_b64[:40])
        new_id = str(uuid4())
        new_element_id = randint(1000, 9999)
        sql = f"""
        INSERT INTO tbl_doc_elements (
            file_id, element_id, element_type, image_base64)
        VALUES 
        ('{new_id}','{new_element_id}','IMAGE','{el_image_b64}')
        RETURNING id;
        """
        # console.print(sql)
        try:
            cur.execute(sql)
            row = cur.fetchone()
            id = row[0]
            console.print(f"ID is {id}")
            conn.commit()
        except Exception as e:
            console.print(e)
    if json_data[i]["type"] == "NarrativeText":

        console.print(f"[dark_orange]---------Item Number TEXT: {i + 1}--------[/]")
        el_text = json_data[i]["text"]
        el_embedding = get_embedding(el_text)

        # console.print(el_embedding)
        new_id = str(uuid4())
        new_element_id = randint(1000, 9999)
        sql = f"""
        INSERT INTO tbl_doc_elements (
            file_id, element_id, element_text, element_type, image_base64, embedding)
        VALUES 
        ('{new_id}','{new_element_id}','{el_text}','NARRATIVE_TEXT',
        '{encoded_image_data}', '{el_embedding}')
        RETURNING id;
        """

        try:
            cur.execute(sql)
            row = cur.fetchone()
            id = row[0]
            console.print(f"ID is {id}")
            conn.commit()
        except Exception as e:
            console.print(e)

Number of items: 12


---------Item Number IMAGE: 1--------

ID is fb705df6-d810-487b-823d-6a44f2150cea

---------Item Number TEXT: 2--------

ID is ef580c88-d1a8-4c69-9e5c-e2a67092d9ad

...

ID is c93895c0-de14-47ba-b60e-34d4bfbdb55b

---------Item Number TEXT: 11--------

ID is bd877dd8-1a14-41e5-85ce-a65c934f8079

---------Item Number TEXT: 12--------

ID is fb66167d-54fe-408e-b086-4d52cbfdf707

In [34]:
cur.close()
conn.close()